In [1]:
from glovar import BRAND_LIST,IPHONE_LIST,STORAGE_LIST
import re

def get_phone_attribute_german(info_txt):
    """
    infomation txt: e.g "Samsung Galaxy S20 5G 128GB Mobile Phone - Grey"
    """
    brand, phone_type, storage_and_memory,color = None,None,None,None
    phone_type = None
    is_5g = False

    info = info_txt.strip().split()

    # get storage_and_memory info
    for storage in STORAGE_LIST:
        if storage in info_txt:
            storage_and_memory = storage
    
    # get brand and phone_type info 
    start,end = 0,0
    if "iphone" in info_txt or "apple" in info_txt:
        brand = "Apple"
        start = info_txt.strip().split().index("iphone")
        if "5g" in info_txt:
            end = info_txt.strip().split().index("5g")
        elif storage_and_memory:
            end = info_txt.strip().split().index(storage_and_memory)
        # for t in IPHONE_LIST:
        #     if t in info_txt:
        #         phone_type = t
        phone_type = " ".join(info[start:end])
    elif "samsung" in info_txt:
        brand = "Samsung"
        start = info_txt.strip().split().index("samsung")
        if "5g" in info_txt:
            end = info_txt.strip().split().index("5g")
        elif storage_and_memory:
            end = info_txt.strip().split().index(storage_and_memory)
        phone_type = " ".join(info[start+1:end])
    elif "xiaomi" in info_txt:
        brand = "Xiaomi"
        start = info_txt.strip().split().index("xiaomi")
        if "5g" in info_txt:
            end = info_txt.strip().split().index("5g")
        elif storage_and_memory:
            end = info_txt.strip().split().index(storage_and_memory)       
        phone_type = " ".join(info[start+1:end])
    elif "oppo" in info_txt:
        brand = "OPPO"
        start = info_txt.strip().split().index("oppo")
        if "5g" in info_txt:
            end = info_txt.strip().split().index("5g")
        elif storage_and_memory:
            end = info_txt.strip().split().index(storage_and_memory)
        phone_type = " ".join(info[start+1:end])
    
    if phone_type and "(" in phone_type:
        phone_type = phone_type.split("(")[0]

    # get is_5g info
    if "5g" in info_txt:
        is_5g = True

    # get color info
    if storage_and_memory:
        color_start = info_txt.strip().split().index(storage_and_memory)
        if "dual" in info_txt:
            color_end = info_txt.strip().split().index("dual")
            color = " ".join(info_txt.strip().split()[color_start+1:color_end])
        else:
            color = " ".join(info_txt.strip().split()[color_start+1:color_start+2])
        
        # print("color:",color)

    return brand, phone_type, storage_and_memory, is_5g, color

def extract_and_save_german(book,sheet,file_name):
    sheet["C1"], sheet["D1"], sheet["E1"], sheet["F1"],sheet["G1"] = "brand", "phone_type", "storage_and_memory", "is_5g", "price"
    sheet["H1"] = "color"
    for i,row in enumerate(sheet.values):
        source_txt, price = row[0],row[1]
        if i > 0 :
            if source_txt and price:
                txt = source_txt.lower()
                txt = txt.strip().replace(" gb","gb")
                txt = txt.strip().replace(" tb","tb")
                price = float(re.findall(r"\d+\.?\d*",price)[0])
                # print(txt, price)

                brand, phone_name, storage_and_memory, is_5g, color = get_phone_attribute_german(txt)
                brand_place = "C" + str(i+1)
                phone_type_place = "D" + str(i+1)
                storage_and_memory_place = "E" + str(i+1)
                is_5g_palce = "F" + str(i+1)
                price_place = "G" + str(i+1)
                color_place = "H" + str(i+1)

                sheet[brand_place] = brand
                sheet[phone_type_place] = phone_name
                sheet[storage_and_memory_place] = storage_and_memory
                sheet[is_5g_palce] = is_5g
                sheet[price_place] = price
                sheet[color_place] = color
            else:
                print("german badcase in line {}!!!".format(i))

    book.save(file_name)

def create_new_sheet_german(book,new_sheet,file_name):
    if "german_test" in book.sheetnames:
        book.remove(book["german_test"])
    ws_1 = book.create_sheet('german_test')
    ws_1["A1"],ws_1["B1"],ws_1["C1"],ws_1["D1"],ws_1["E1"] = "BRAND", "TYPE", "RAM_AND_STORAGE", "TIME", "PRICE" 
    ws_1["F1"],ws_1["G1"],ws_1["H1"],ws_1["I1"],ws_1["J1"],ws_1["K1"],ws_1["L1"] = "GERMAN","FRANCE","UK","ITALY","SPAIN","FINLAND","CZECH"

    german_phone = set()

    k = 2
    for i,row in enumerate(new_sheet.values):
        if i > 0:
            brand, phone_type, storage, is_5g, price, color = row[2],row[3],row[4],bool(row[5]),row[1],row[6]
            # print(brand, phone_type, storage, is_5g, price,color)
            if brand and phone_type and storage and price:
                price = str(price.split(".")[0])
                if is_5g:
                    phone_type = phone_type.strip() + " 5g "
                
                new_info = [brand, phone_type, storage]
                unique_phone_id  = " ".join(new_info)
                # unique_phone_id = brand + " " + phone_type + " " + storage +  " " + str(price)
                # if unique_phone_id not in german_phone:
                    # print(unique_phone_id,price)
                brand_place = "A" + str(k)
                phone_type_place = "B" + str(k)
                storage_and_memory_place = "C" + str(k)
                price_place = "F" + str(k)
                
                ws_1[brand_place] = brand
                ws_1[phone_type_place] = phone_type
                ws_1[storage_and_memory_place] = storage
                ws_1[price_place] = price
                k += 1
                
                # german_phone.add(unique_phone_id)
    # print(len(german_phone))

    # for t in sorted(list(german_phone)):
    #     print(t)

    book.save(file_name)